In [1]:
#LIBRERIAS
import pandas as pd
import numpy as np
import mysql.connector

./EDA_UCI/dataset/

In [2]:
# Conectar a la base de datos 
cnx = mysql.connector.connect(user='administrador123', password='pasword123.',
                              host='proyectdb.mysql.database.azure.com', database='proyectdb', connect_timeout=174000)
cursor = cnx.cursor()


In [3]:
# Crear la tabla en la base de datos
create_table_query = """
CREATE TABLE admissions2 (
  ROW_ID INT NOT NULL,
  SUBJECT_ID INT NOT NULL,
  HADM_ID INT NOT NULL,
  ADMITTIME DATE,
  DISCHTIME DATE,
  DEATHTIME DATE,
  ADMISSION_TYPE VARCHAR(50) NOT NULL,
  ADMISSION_LOCATION VARCHAR(50),
  DISCHARGE_LOCATION VARCHAR(50),
  INSURANCE VARCHAR(255) NOT NULL,
  LANGUAGE VARCHAR(10),
  RELIGION VARCHAR(50),
  MARITAL_STATUS VARCHAR(50),
  ETHNICITY VARCHAR(200),
  EDREGTIME DATE,
  EDOUTTIME DATE,
  DIAGNOSIS VARCHAR(300),
  HOSPITAL_EXPIRE_FLAG TINYINT NOT NULL,
  HAS_CHARTEVENTS_DATA TINYINT NOT NULL,
  PRIMARY KEY (ROW_ID)
) ENGINE=InnoDB
"""
cursor.execute(create_table_query)

In [4]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/ADMISSIONS.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/ADMISSIONS.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [5]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [6]:
# Formateamos las columnas de fecha a un formato que solo incluye la fecha


df['admittime'] = pd.to_datetime(df['admittime']).dt.date
df['dischtime'] = pd.to_datetime(df['dischtime']).dt.date
df['edregtime'] = pd.to_datetime(df['edregtime']).dt.date
df['edouttime'] = pd.to_datetime(df['edouttime']).dt.date
df['deathtime'] = pd.to_datetime(df['deathtime']).dt.date.replace(pd.NaT, None)


In [7]:
#cargar los datos a la tabla
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO admissions2
    (ROW_ID, SUBJECT_ID, HADM_ID, ADMITTIME, DISCHTIME, DEATHTIME, ADMISSION_TYPE, ADMISSION_LOCATION, DISCHARGE_LOCATION, INSURANCE, LANGUAGE, RELIGION, MARITAL_STATUS, ETHNICITY, EDREGTIME, EDOUTTIME, DIAGNOSIS, HOSPITAL_EXPIRE_FLAG, HAS_CHARTEVENTS_DATA)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA ICUSTDAY

In [8]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE ICUSTAYS2 (
  ROW_ID INT,
  SUBJECT_ID INT,
  HADM_ID INT,
  ICUSTAY_ID INT,
  DBSOURCE VARCHAR(20),
  FIRST_CAREUNIT VARCHAR(20),
  LAST_CAREUNIT VARCHAR(20),
  FIRST_WARDID SMALLINT,
  LAST_WARDID SMALLINT,
  INTIME DATE,
  OUTTIME DATE,
  LOS DOUBLE(10,2),
  PRIMARY KEY (ROW_ID)
) ENGINE=InnoDB;
'''
cursor.execute(create_table_query)


In [9]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/ICUSTAYS.csv')

# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/ICUSTAYS.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [10]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [11]:
#cambio del formato de las fechas
df['intime'] = pd.to_datetime(df['intime']).dt.date.replace(pd.NaT, None)
df['outtime'] = pd.to_datetime(df['outtime']).dt.date.replace(pd.NaT, None)

In [12]:
#ingreso de datos a la tabla
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO ICUSTAYS2
    (ROW_ID, SUBJECT_ID, HADM_ID, ICUSTAY_ID, DBSOURCE, FIRST_CAREUNIT, LAST_CAREUNIT, FIRST_WARDID, LAST_WARDID, INTIME, OUTTIME, LOS)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA PATIENTS

In [13]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE PATIENT2 (
  ROW_ID INT,
  SUBJECT_ID INT,
  GENDER VARCHAR(5),
  DOB DATE,
  DOD DATE,
  DOD_HOSP DATE,
  DOD_SSN DATE,
  EXPIRE_FLAG VARCHAR(5)
) ENGINE=InnoDB;
'''
cursor.execute(create_table_query)


In [14]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/PATIENTS.csv')

In [15]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [16]:
# Convertir columnas de fecha a objetos datetime y reemplazar los valores NaT con None
df['dob'] = pd.to_datetime(df['dob']).dt.date.replace(pd.NaT, None)
df['dod'] = pd.to_datetime(df['dod']).dt.date.replace(pd.NaT, None)
df['dod_hosp'] = pd.to_datetime(df['dod_hosp']).dt.date.replace(pd.NaT, None)
df['dod_ssn'] = pd.to_datetime(df['dod_ssn']).dt.date.replace(pd.NaT, None)

In [17]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO PATIENT2
    (ROW_ID, SUBJECT_ID, GENDER, DOB, DOD, DOD_HOSP, DOD_SSN, EXPIRE_FLAG)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA CPTEVENTS

In [18]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE CPTEVENTS2 (
  ROW_ID INT PRIMARY KEY,
  SUBJECT_ID INT,
  HADM_ID INT,
  COSTCENTER VARCHAR(10),
  CHARTDATE DATE,
  CPT_CD VARCHAR(10),
  CPT_NUMBER INT,
  CPT_SUFFIX VARCHAR(5),
  TICKET_ID_SEQ INT,
  SECTIONHEADER VARCHAR(50),
  SUBSECTIONHEADER VARCHAR(300),
  DESCRIPTION VARCHAR(200)
) ENGINE=InnoDB;
'''
cursor.execute(create_table_query)


In [19]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/CPTEVENTS.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/CPTEVENTS.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)


In [20]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [21]:
df['chartdate'] = pd.to_datetime(df['chartdate']).dt.date.replace(pd.NaT, None)

In [22]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO CPTEVENTS2
    (ROW_ID,SUBJECT_ID,HADM_ID,COSTCENTER,CHARTDATE,CPT_CD,CPT_NUMBER,CPT_SUFFIX,TICKET_ID_SEQ,SECTIONHEADER,SUBSECTIONHEADER,DESCRIPTION)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA PROCEDUREEVENTS_MV

In [23]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE PROCEDUREEVENTS_MV2 (
  ROW_ID INT NOT NULL,
  SUBJECT_ID INT NOT NULL,
  HADM_ID INT NOT NULL,
  ICUSTAY_ID INT,
  STARTTIME DATE,
  ENDTIME DATE,
  ITEMID INT,
  VALUE DOUBLE PRECISION,
  VALUEUOM VARCHAR(30),
  LOCATION VARCHAR(30),
  LOCATIONCATEGORY VARCHAR(30),
  STORETIME DATE,
  CGID INT,
  ORDERID INT,
  LINKORDERID INT,
  ORDERCATEGORYNAME VARCHAR(100),
  SECONDARYORDERCATEGORYNAME VARCHAR(100),
  ORDERCATEGORYDESCRIPTION VARCHAR(50),
  ISOPENBAG SMALLINT,
  CONTINUEINNEXTDEPT SMALLINT,
  CANCELREASON SMALLINT,
  STATUSDESCRIPTION VARCHAR(30),
  COMMENTS_EDITEDBY VARCHAR(30),
  COMMENTS_CANCELEDBY VARCHAR(30),
  COMMENTS_DATE DATE
) 
'''
cursor.execute(create_table_query)

In [24]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/PROCEDUREEVENTS_MV.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/PROCEDUREEVENTS_MV.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [25]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [26]:
df['starttime'] = pd.to_datetime(df['starttime']).dt.date
df['endtime'] = pd.to_datetime(df['endtime']).dt.date
df['storetime'] = pd.to_datetime(df['storetime']).dt.date
df['comments_date'] = pd.to_datetime(df['comments_date']).dt.date

In [27]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO PROCEDUREEVENTS_MV2
    (ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,VALUE,VALUEUOM,LOCATION,LOCATIONCATEGORY,STORETIME,CGID,ORDERID,LINKORDERID,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCATEGORYDESCRIPTION,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA PROCEDURES_ICD

In [28]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE PRESCRIPTIONS2 (
  ROW_ID INT,
  SUBJECT_ID INT,
  HADM_ID INT,
  ICUSTAY_ID INT,
  STARTDATE DATE,
  ENDDATE DATE,
  DRUG_TYPE VARCHAR(100),
  DRUG VARCHAR(100),
  DRUG_NAME_POE VARCHAR(100),
  DRUG_NAME_GENERIC VARCHAR(100),
  FORMULARY_DRUG_CD VARCHAR(120),
  GSN VARCHAR(200),
  NDC VARCHAR(120),
  PROD_STRENGTH VARCHAR(120),
  DOSE_VAL_RX VARCHAR(120),
  DOSE_UNIT_RX VARCHAR(120),
  FORM_VAL_DISP VARCHAR(120),
  FORM_UNIT_DISP VARCHAR(120),
  ROUTE VARCHAR(120)
) ENGINE=InnoDB;
'''
cursor.execute(create_table_query)

In [29]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/PRESCRIPTIONS.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/PRESCRIPTIONS.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [30]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [31]:
df['startdate'] = pd.to_datetime(df['startdate']).dt.date.replace(pd.NaT, None)
df['enddate'] = pd.to_datetime(df['enddate']).dt.date.replace(pd.NaT, None)


In [32]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO PRESCRIPTIONS2
    (ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN, NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA LABEVENTS

In [34]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE LABEVENTS3 (
  ROW_ID INT,
  SUBJECT_ID INT,
  HADM_ID INT,
  ITEMID INT,
  CHARTTIME DATE,
  VALUE VARCHAR(200),
  VALUENUM DOUBLE PRECISION,
  VALUEUOM VARCHAR(20),
  FLAG VARCHAR(20)
) 
'''
cursor.execute(create_table_query)

In [35]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/LABEVENTS.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/LABEVENTS.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [36]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [37]:
df['charttime']  = pd.to_datetime(df['charttime']).dt.date.replace(pd.NaT, None)


In [38]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO LABEVENTS3
    (ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA CALLOUTS

In [39]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE CALLOUT2 (
  ROW_ID INT,
  SUBJECT_ID INT,
  HADM_ID INT,
  SUBMIT_WARDID INT,
  SUBMIT_CAREUNIT VARCHAR(15),
  CURR_WARDID INT,
  CURR_CAREUNIT VARCHAR(15),
  CALLOUT_WARDID INT,
  CALLOUT_SERVICE VARCHAR(10),
  REQUEST_TELE SMALLINT,
  REQUEST_RESP SMALLINT,
  REQUEST_CDIFF SMALLINT,
  REQUEST_MRSA SMALLINT,
  REQUEST_VRE SMALLINT,
  CALLOUT_STATUS VARCHAR(20),
  CALLOUT_OUTCOME VARCHAR(20),
  DISCHARGE_WARDID INT,
  ACKNOWLEDGE_STATUS VARCHAR(20),
  CREATETIME DATE,
  UPDATETIME DATE,
  ACKNOWLEDGETIME DATE,
  OUTCOMETIME DATE,
  FIRSTRESERVATIONTIME DATE,
  CURRENTRESERVATIONTIME DATE
) ENGINE=InnoDB;
'''
cursor.execute(create_table_query)

In [40]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/CALLOUT.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/CALLOUT.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [41]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [42]:
df['createtime']  = pd.to_datetime(df['createtime']).dt.date.replace(pd.NaT, None)
df['updatetime']  = pd.to_datetime(df['updatetime']).dt.date.replace(pd.NaT, None)
df['acknowledgetime']  = pd.to_datetime(df['acknowledgetime']).dt.date.replace(pd.NaT, None)
df['outcometime']  = pd.to_datetime(df['outcometime']).dt.date.replace(pd.NaT, None)
df['firstreservationtime']  = pd.to_datetime(df['firstreservationtime']).dt.date.replace(pd.NaT, None)
df['currentreservationtime']  = pd.to_datetime(df['currentreservationtime']).dt.date.replace(pd.NaT, None)

In [43]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO CALLOUT2
    (ROW_ID,SUBJECT_ID,HADM_ID,SUBMIT_WARDID,SUBMIT_CAREUNIT,CURR_WARDID,CURR_CAREUNIT,CALLOUT_WARDID,CALLOUT_SERVICE,REQUEST_TELE,REQUEST_RESP,REQUEST_CDIFF,REQUEST_MRSA,REQUEST_VRE,CALLOUT_STATUS,CALLOUT_OUTCOME,DISCHARGE_WARDID,ACKNOWLEDGE_STATUS,CREATETIME,UPDATETIME,ACKNOWLEDGETIME,OUTCOMETIME,FIRSTRESERVATIONTIME,CURRENTRESERVATIONTIME)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA TRANSFER

In [44]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE TRANSFERS2 (
  ROW_ID INT,
  SUBJECT_ID INT,
  HADM_ID INT,
  ICUSTAY_ID INT,
  DBSOURCE VARCHAR(20),
  EVENTTYPE VARCHAR(20),
  PREV_CAREUNIT VARCHAR(20),
  CURR_CAREUNIT VARCHAR(20),
  PREV_WARDID SMALLINT,
  CURR_WARDID SMALLINT,
  INTIME DATE,
  OUTTIME DATE,
  LOS INT
) ENGINE=InnoDB;
'''
cursor.execute(create_table_query)

In [45]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/TRANSFERS.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/TRANSFERS.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [46]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [47]:
df['intime']  = pd.to_datetime(df['intime']).dt.date.replace(pd.NaT, None)
df['outtime']  = pd.to_datetime(df['outtime']).dt.date.replace(pd.NaT, None)


In [48]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO TRANSFERS2
    (ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,EVENTTYPE,PREV_CAREUNIT,CURR_CAREUNIT,PREV_WARDID,CURR_WARDID,INTIME,OUTTIME,LOS)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA SERVICES

In [49]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE SERVICES2 (
  ROW_ID INT,
  SUBJECT_ID INT,
  HADM_ID INT,
  TRANSFERTIME DATE,
  PREV_SERVICE VARCHAR(20),
  CURR_SERVICE VARCHAR(20)
) ENGINE=InnoDB;
'''
cursor.execute(create_table_query)

In [50]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/SERVICES.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/SERVICES.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [51]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [52]:
df['transfertime']  = pd.to_datetime(df['transfertime']).dt.date.replace(pd.NaT, None)


In [53]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO SERVICES2
    (ROW_ID,SUBJECT_ID,HADM_ID,TRANSFERTIME,PREV_SERVICE,CURR_SERVICE)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA CAREGIVERS

In [54]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE CAREGIVERS2 (
  ROW_ID INTEGER,
  CGID INTEGER,
  LABEL VARCHAR(15),
  DESCRIPTION VARCHAR(30)
) 
'''
cursor.execute(create_table_query)

In [55]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/CAREGIVERS.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/CAREGIVERS.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [56]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [57]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO CAREGIVERS2
    (ROW_ID,CGID,LABEL,DESCRIPTION)
    VALUES (%s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA D_CPT

In [58]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE D_CPT2 (
  ROW_ID INT,
  CATEGORY SMALLINT,
  SECTIONRANGE VARCHAR(100),
  SECTIONHEADER VARCHAR(50),
  SUBSECTIONRANGE VARCHAR(100),
  SUBSECTIONHEADER VARCHAR(300),
  CODESUFFIX VARCHAR(5),
  MINCODEINSUBSECTION INT,
  MAXCODEINSUBSECTION INT
) 
'''
cursor.execute(create_table_query)

In [59]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/D_CPT.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/D_CPT.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [60]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [61]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO D_CPT2
    (ROW_ID,CATEGORY,SECTIONRANGE,SECTIONHEADER,SUBSECTIONRANGE,SUBSECTIONHEADER,CODESUFFIX,MINCODEINSUBSECTION,MAXCODEINSUBSECTION)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA D_ICD_DIAGNOSES

In [62]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE D_ICD_DIAGNOSES2 (
  ROW_ID INT,
  ICD9_CODE VARCHAR(10),
  SHORT_TITLE VARCHAR(50),
  LONG_TITLE VARCHAR(300)
) 
'''
cursor.execute(create_table_query)

In [63]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/D_ICD_DIAGNOSES.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/D_ICD_DIAGNOSES.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [64]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [65]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO D_ICD_DIAGNOSES2
    (ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE)
    VALUES (%s, %s, %s, %s)

    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA D_ICD_PROCEDURES

In [66]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE D_ICD_PROCEDURES2 (
  ROW_ID INT,
  ICD9_CODE VARCHAR(10),
  SHORT_TITLE VARCHAR(50),
  LONG_TITLE VARCHAR(300)
) 
'''
cursor.execute(create_table_query)

In [67]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/D_ICD_PROCEDURES.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/D_ICD_PROCEDURES.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [68]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [69]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO D_ICD_PROCEDURES2
    (ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE)
    VALUES (%s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA D_ITEMS

In [70]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE D_ITEMS2 (
  ROW_ID INT,
  ITEMID INT,
  LABEL VARCHAR(200),
  ABBREVIATION VARCHAR(100),
  DBSOURCE VARCHAR(20),
  LINKSTO VARCHAR(50),
  CATEGORY VARCHAR(100),
  UNITNAME VARCHAR(100),
  PARAM_TYPE VARCHAR(30),
  CONCEPTID INT
) 
'''
cursor.execute(create_table_query)

In [71]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/D_ITEMS.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/D_ITEMS.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [72]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [73]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO D_ITEMS2
    (ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA DE D_LABITEMS.csv

In [74]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE D_LABITEMS2 (
  ROW_ID INT,
  ITEMID INT,
  LABEL VARCHAR(100),
  FLUID VARCHAR(100),
  CATEGORY VARCHAR(100),
  LOINC_CODE VARCHAR(100)
) 
'''
cursor.execute(create_table_query)

In [75]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/D_LABITEMS.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/D_LABITEMS.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [76]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [77]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO D_LABITEMS2
    (ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA DIAGNOSES_ICD

In [78]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE DIAGNOSES_ICD2 (
  ROW_ID INT NOT NULL,
  SUBJECT_ID INT NOT NULL,
  HADM_ID INT NOT NULL,
  SEQ_NUM INT,
  ICD9_CODE VARCHAR(10)
) 
'''
cursor.execute(create_table_query)

In [79]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/DIAGNOSES_ICD.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/DIAGNOSES_ICD.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [80]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [81]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO DIAGNOSES_ICD2
    (ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE)
    VALUES (%s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA DRGCODES

In [82]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE DDRGCODES2 (
  ROW_ID INT, 
  SUBJECT_ID INT, 
  HADM_ID INT, 
  DRG_TYPE VARCHAR(20),
  DRG_CODE VARCHAR(20), 
  DESCRIPTION VARCHAR(300),
  DRG_SEVERITY SMALLINT,
  DRG_MORTALITY SMALLINT
) 
'''
cursor.execute(create_table_query)

In [83]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/DRGCODES.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/DRGCPDES.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [84]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [85]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO DDRGCODES2
    (ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA INPUTEVENTS_CV

In [86]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE INPUTEVENTS_CV2 (
  ROW_ID INT,
  SUBJECT_ID INT,
  HADM_ID INT,
  ICUSTAY_ID INT,
  CHARTTIME DATE,
  ITEMID INT,
  AMOUNT DOUBLE PRECISION,
  AMOUNTUOM VARCHAR(30),
  RATE DOUBLE PRECISION,
  RATEUOM VARCHAR(30),
  STORETIME DATE,
  CGID BIGINT,
  ORDERID BIGINT,
  LINKORDERID BIGINT,
  STOPPED VARCHAR(30),
  NEWBOTTLE INT,
  ORIGINALAMOUNT DOUBLE PRECISION,
  ORIGINALAMOUNTUOM VARCHAR(30),
  ORIGINALROUTE VARCHAR(30),
  ORIGINALRATE DOUBLE PRECISION,
  ORIGINALRATEUOM VARCHAR(30),
  ORIGINALSITE VARCHAR(30)
) 
'''
cursor.execute(create_table_query)

In [87]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/INPUTEVENTS_CV.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/IMPUTEVENTS_CV.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

C:\Users\W10-PC\AppData\Local\Temp\ipykernel_4604\3851768261.py:2: DtypeWarning: Columns (17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('INPUTEVENTS_CV.csv')


In [88]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [89]:
df['charttime']  = pd.to_datetime(df['charttime']).dt.date.replace(pd.NaT, None)
df['storetime']  = pd.to_datetime(df['storetime']).dt.date.replace(pd.NaT, None)

In [90]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO INPUTEVENTS_CV2
    (ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,STORETIME,CGID,ORDERID,LINKORDERID,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA INPUTEVENTS_MV

In [91]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE INPUTEVENTS_MV2 (
  ROW_ID INT,
  SUBJECT_ID INT,
  HADM_ID INT,
  ICUSTAY_ID INT,
  STARTTIME DATE,
  ENDTIME DATE,
  ITEMID INT,
  AMOUNT DOUBLE PRECISION,
  AMOUNTUOM VARCHAR(30),
  RATE DOUBLE PRECISION,
  RATEUOM VARCHAR(30),
  STORETIME DATE,
  CGID BIGINT,
  ORDERID BIGINT,
  LINKORDERID BIGINT,
  ORDERCATEGORYNAME VARCHAR(100),
  SECONDARYORDERCATEGORYNAME VARCHAR(100),
  ORDERCOMPONENTTYPEDESCRIPTION VARCHAR(200),
  ORDERCATEGORYDESCRIPTION VARCHAR(50),
  PATIENTWEIGHT DOUBLE PRECISION,
  TOTALAMOUNT DOUBLE PRECISION,
  TOTALAMOUNTUOM VARCHAR(50),
  ISOPENBAG SMALLINT,
  CONTINUEINNEXTDEPT SMALLINT,
  CANCELREASON SMALLINT,
  STATUSDESCRIPTION VARCHAR(30),
  COMMENTS_STATUS VARCHAR(30),
  COMMENTS_TITLE VARCHAR(100),
  COMMENTS_DATE DATE,
  ORIGINALAMOUNT DOUBLE PRECISION,
  ORIGINALRATE DOUBLE PRECISION
) 
'''
cursor.execute(create_table_query)

In [92]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/INPUTEVENTS_MV.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/IMPUTEVENTS_MV.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [93]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [94]:
df['endtime']  = pd.to_datetime(df['endtime']).dt.date.replace(pd.NaT, None)
df['storetime']  = pd.to_datetime(df['storetime']).dt.date.replace(pd.NaT, None)
df['starttime']  = pd.to_datetime(df['starttime']).dt.date.replace(pd.NaT, None)
df['comments_date']  = pd.to_datetime(df['comments_date']).dt.date.replace(pd.NaT, None)

In [95]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO INPUTEVENTS_MV2
    (ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,STORETIME,CGID,ORDERID,LINKORDERID,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCOMPONENTTYPEDESCRIPTION,ORDERCATEGORYDESCRIPTION,PATIENTWEIGHT,TOTALAMOUNT,TOTALAMOUNTUOM,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_STATUS,COMMENTS_TITLE,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA MICROBIOLOGYEVENTS

In [96]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE MICROBIOLOGYEVENTS2 (
  ROW_ID INT,
  SUBJECT_ID INT,
  HADM_ID INT,
  CHARTDATE DATE,
  CHARTTIME DATE,
  SPEC_ITEMID INT,
  SPEC_TYPE_DESC VARCHAR(100),
  ORG_ITEMID INT,
  ORG_NAME VARCHAR(100),
  ISOLATE_NUM SMALLINT,
  AB_ITEMID INT,
  AB_NAME VARCHAR(30),
  DILUTION_TEXT VARCHAR(10),
  DILUTION_COMPARISON VARCHAR(20),
  DILUTION_VALUE DOUBLE PRECISION,
  INTERPRETATION VARCHAR(5)
) 
'''
cursor.execute(create_table_query)

In [97]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/MICROBIOLOGYEVENTS.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/MICROBIOLOGYEVENTS.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [98]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [99]:

df['chartdate']  = pd.to_datetime(df['chartdate']).dt.date.replace(pd.NaT, None)
df['charttime']  = pd.to_datetime(df['charttime']).dt.date.replace(pd.NaT, None)

In [100]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO MICROBIOLOGYEVENTS2
    (ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA OUTPUTEVENTS

In [101]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE OUTPUTEVENTS2 (
    ROW_ID INTEGER,
    SUBJECT_ID INTEGER,
    HADM_ID INTEGER,
    ICUSTAY_ID INTEGER,
    CHARTTIME DATE,
    ITEMID INTEGER,
    VALUE DOUBLE PRECISION,
    VALUEUOM VARCHAR(30),
    STORETIME DATE,
    CGID BIGINT,
    STOPPED VARCHAR(30),
    NEWBOTTLE INTEGER,
    ISERROR SMALLINT
) 
'''
cursor.execute(create_table_query)

In [102]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/OUTPUTEVENTS.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/OUTPUTEVENTS.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [103]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [104]:

df['storetime']  = pd.to_datetime(df['storetime']).dt.date.replace(pd.NaT, None)
df['charttime']  = pd.to_datetime(df['charttime']).dt.date.replace(pd.NaT, None)

In [105]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO OUTPUTEVENTS2
    (ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA DATETIMEEVENTS

In [106]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE DATETIMEEVENTS2 (
    ROW_ID INT,
    SUBJECT_ID INT,
    HADM_ID INT,
    ICUSTAY_ID INT,
    ITEMID INT,
    CHARTTIME DATE,
    STORETIME DATE,
    CGID INT,
    VALUE DATE,
    VALUEUOM VARCHAR(50),
    WARNING SMALLINT,
    ERROR SMALLINT,
    RESULTSTATUS VARCHAR(50),
    STOPPED VARCHAR(50)
) 
'''
cursor.execute(create_table_query)

In [107]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/DATETIMEEVENTS.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/DATETIMEEVENTS.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [108]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [109]:

df['storetime']  = pd.to_datetime(df['storetime']).dt.date.replace(pd.NaT, None)
df['charttime']  = pd.to_datetime(df['charttime']).dt.date.replace(pd.NaT, None)
df['value']  = pd.to_datetime(df['value']).dt.date.replace(pd.NaT, None)


In [110]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO DATETIMEEVENTS2
    (ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA D_CPT EVENTS

In [112]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE D_CPT3 (
    ROW_ID INT,
    CATEGORY SMALLINT,
    SECTIONRANGE VARCHAR(100),
    SECTIONHEADER VARCHAR(50),
    SUBSECTIONRANGE VARCHAR(100),
    SUBSECTIONHEADER VARCHAR(300),
    CODESUFFIX VARCHAR(5),
    MINCODEINSUBSECTION	INT,
    MAXCODEINSUBSECTION	INT
) 
'''
cursor.execute(create_table_query)

In [113]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/D_CPT.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/D_CPT.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [114]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [115]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO D_CPT2
    (ROW_ID,CATEGORY,SECTIONRANGE,SECTIONHEADER,SUBSECTIONRANGE,SUBSECTIONHEADER,CODESUFFIX,MINCODEINSUBSECTION,MAXCODEINSUBSECTION)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s)
    """
    data = tuple(row.values)
    cursor.execute(insert_query, data)
    cnx.commit()

TABLA PROCEDUREEVENTS_ICD

In [126]:
# Definir la consulta SQL para crear la tabla
create_table_query = '''
CREATE TABLE PROCEDURES_ICD2 (
    ROW_ID INTEGER NOT NULL,
    SUBJECT_ID INTEGER NOT NULL,
    HADM_ID INTEGER NOT NULL,
    SEQ_NUM INTEGER,
    ICD9_CODE VARCHAR(10)
) 
'''
cursor.execute(create_table_query)

In [117]:
# Leer la tabla "admissions" en un DataFrame de pandas
df = pd.read_csv('./EDA_UCI/dataset/PROCEDURES_ICD.csv')
# Seleccionar el 20% inferior del DataFrame
n = int(len(df) * 0.2)
df1_selected = df.tail(n)
#guardo la parte seleccionada
df1_selected.to_csv('RECORTES/PROCEDURES_ICD.csv')

# Calcular el número de filas a eliminar (20% del total)
n = int(len(df) * 0.2)

# Obtener los índices de las filas a eliminar (los últimos n)
idx_to_drop = df.tail(n).index

# Eliminar las filas del DataFrame original
df = df.drop(idx_to_drop)

In [127]:
# Convertir los valores nan a None
df = df.where(pd.notnull(df), None)

texto_cols = df.select_dtypes(include=['string']).columns
df[texto_cols] = df[texto_cols].fillna('SD')

num_cols = df.select_dtypes(include=['int', 'float']).columns
df[num_cols] = df[num_cols].fillna(0)

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405 entries, 0 to 404
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   row_id      405 non-null    int64
 1   subject_id  405 non-null    int64
 2   hadm_id     405 non-null    int64
 3   seq_num     405 non-null    int64
 4   icd9_code   405 non-null    int64
dtypes: int64(5)
memory usage: 15.9 KB


In [130]:
for index, row in df.iterrows():
    insert_query = """
    INSERT INTO PROCEDURES_ICD2
    (ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE)
    VALUES (%s, %s, %s, %s, %s)
    """
    data = data = tuple(int(val) if isinstance(val, np.int64) else val for val in row.values)

    cursor.execute(insert_query, data)
    cnx.commit()

In [ ]:
# Guardar los cambios y cerrar la conexión
cnx.commit()
cursor.close()
cnx.close()